# Processa todos os datasets da saída

Divisão: 80% treino e 20% validação | kfold com 5 folds, pegando os datasets originais para validação e os sintéticos para treino. 

# 1) Processamento dos dados originais

In [2]:
import os
import pandas as pd
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.utils.validation import check_X_y
from joblib import Parallel, delayed
from tqdm import tqdm
import numpy as np

# Função para carregar todos os arquivos .txt de todas as subpastas de um diretório
def load_datasets_from_directories(base_directories):
    datasets = {}
    for base_directory in base_directories:
        for root, _, files in os.walk(base_directory):
            for file in files:
                filepath = os.path.join(root, file)
                filename, ext = os.path.splitext(file)

                if ext != '.txt':  # Ignorar arquivos que não sejam .txt
                    continue

                try:
                    with open(filepath, 'r') as f:
                        first_line = f.readline()
                        delimiter = ',' if ',' in first_line else ('\t' if '\t' in first_line else ' ')
                    datasets[filepath] = pd.read_csv(filepath, delimiter=delimiter)
                except Exception as e:
                    print(f"Erro ao carregar {filepath}: {e}")

    return datasets

# Função para processar um único modelo com KFold
def process_model_kfold(X, y, model_name, model, param_grid, repetitions, n_splits=5):
    metrics = {'mse': [], 'rmse': [], 'r2': [], 'mae': []}

    for _ in range(repetitions):
        kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
        grid = GridSearchCV(model, param_grid[model_name], cv=kf, scoring='neg_mean_squared_error', error_score='raise')

        try:
            grid.fit(X, y)
            best_model = grid.best_estimator_

            # Avaliação com KFold
            for train_index, test_index in kf.split(X):
                X_train, X_test = X[train_index], X[test_index]
                y_train, y_test = y[train_index], y[test_index]

                best_model.fit(X_train, y_train)
                y_pred = best_model.predict(X_test)

                # Cálculo das métricas
                mse = mean_squared_error(y_test, y_pred)
                rmse = np.sqrt(mse)
                r2 = r2_score(y_test, y_pred)
                mae = mean_absolute_error(y_test, y_pred)

                metrics['mse'].append(mse)
                metrics['rmse'].append(rmse)
                metrics['r2'].append(r2)
                metrics['mae'].append(mae)

        except Exception as e:
            print(f"{model_name}: Falha no treinamento - {e}")
            continue

    return {
        'mse': np.mean(metrics['mse']),
        'rmse': np.mean(metrics['rmse']),
        'r2': np.mean(metrics['r2']),
        'mae': np.mean(metrics['mae']),
        'best_params': grid.best_params_ if 'grid' in locals() else {}
    }

# Função para treinar e testar modelos de regressão
def test_regression_models(data, repetitions=30):
    models = {
        'LinearRegression': LinearRegression(),
        'Ridge': Ridge(),
        'RandomForest': RandomForestRegressor(),
    }

    param_grid = {
        'LinearRegression': {},
        'Ridge': {'alpha': [0.1, 1, 10]},
        'RandomForest': {'n_estimators': [50, 100, 200]},
    }

    for filepath, df in tqdm(data.items(), desc="Processando datasets"):
        output_directory = os.path.dirname(filepath)
        output_filename = os.path.splitext(os.path.basename(filepath))[0] + '_resultados_processamento_simples.xlsx'
        output_path = os.path.join(output_directory, output_filename)

        df = df.dropna()

        if df.shape[0] < 2 or df.shape[1] < 2:
            print(f"{filepath}: Dataset insuficiente para treino. Tamanho: {df.shape}")
            continue

        X = df.iloc[:, :-1].values
        y = df.iloc[:, -1].values

        try:
            X, y = check_X_y(X, y)
        except ValueError as e:
            print(f"{filepath}: Erro de validação dos dados - {e}")
            continue

        # Processar cada modelo em paralelo
        results = {}
        model_results = Parallel(n_jobs=-1)(
            delayed(process_model_kfold)(X, y, model_name, model, param_grid, repetitions)
            for model_name, model in models.items()
        )

        # Armazenar resultados
        for model_name, metrics in zip(models.keys(), model_results):
            results[model_name] = metrics

        results_df = pd.DataFrame.from_dict(results, orient='index')

        try:
            results_df.to_excel(output_path)
            print(f"Resultados salvos em '{output_path}'.")
        except Exception as e:
            print(f"Erro ao salvar os resultados: {e}")

base_directories = [
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao',
]

# Carregar os datasets de todas as subpastas
data = load_datasets_from_directories(base_directories)

# Processar os datasets e salvar resultados
if data:
    test_regression_models(data, repetitions=30)

Processando datasets:   4%|▎         | 1/28 [20:01<9:00:31, 1201.17s/it]

Resultados salvos em 'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao\1-tratamento\tratamento_china_resultados_processamento_simples.xlsx'.


Processando datasets:   7%|▋         | 2/28 [37:11<7:57:04, 1100.95s/it]

Resultados salvos em 'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao\1-tratamento\tratamento_cocomo81_resultados_processamento_simples.xlsx'.


Processando datasets:  11%|█         | 3/28 [46:36<5:56:39, 855.98s/it] 

...

Resultados salvos em 'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao\MIMIC-75\tratamento_cocomo81_resultados_processamento_simples.xlsx'.


Processando datasets:  96%|█████████▋| 27/28 [9:29:10<21:17, 1277.02s/it]

Resultados salvos em 'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao\MIMIC-75\tratamento_desharnais_resultados_processamento_simples.xlsx'.


Processando datasets: 100%|██████████| 28/28 [10:05:01<00:00, 1296.47s/it]

Resultados salvos em 'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao\MIMIC-75\tratamento_maxwell_resultados_processamento_simples.xlsx'.


# 2) dados sintéticos | usa dados sintéticos para treino e dados originais para teste.

Usa 5 folds, sendo 4 com dados sintéticos e 1 com dados originais.

In [66]:
import os
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from joblib import Parallel, delayed
from tqdm import tqdm
import numpy as np

# Função para carregar datasets
def load_datasets(path):
    datasets = {}
    for root, _, files in os.walk(path):
        for file in files:
            if file.endswith('.txt'):
                file_path = os.path.join(root, file)
                try:
                    datasets[file] = pd.read_csv(file_path, delimiter=None, engine='python')
                except Exception as e:
                    print(f"Erro ao carregar {file}: {e}")
    return datasets

# Função para processar o modelo com KFold
def process_model_kfold(train_data, test_data, model_name, model, repetitions=30, n_splits=5):
    metrics = {'mse': [], 'rmse': [], 'r2': [], 'mae': []}
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    X_train = train_data.iloc[:, :-1].values
    y_train = train_data.iloc[:, -1].values
    X_test = test_data.iloc[:, :-1].values
    y_test = test_data.iloc[:, -1].values

    for _ in range(repetitions):
        for train_index, val_index in kf.split(X_train):
            X_fold_train, X_fold_val = X_train[train_index], X_train[val_index]
            y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]

            try:
                model.fit(X_fold_train, y_fold_train)
                y_pred = model.predict(X_test)  # Validação com dados originais
                mse = mean_squared_error(y_test, y_pred)
                rmse = np.sqrt(mse)
                r2 = r2_score(y_test, y_pred)
                mae = mean_absolute_error(y_test, y_pred)

                metrics['mse'].append(mse)
                metrics['rmse'].append(rmse)
                metrics['r2'].append(r2)
                metrics['mae'].append(mae)
            except Exception as e:
                print(f"Erro ao treinar/avaliar modelo {model_name}: {e}")

    return {
        'mse': np.mean(metrics['mse']),
        'rmse': np.mean(metrics['rmse']),
        'r2': np.mean(metrics['r2']),
        'mae': np.mean(metrics['mae']),
        'hyperparameters': model.get_params() if hasattr(model, 'get_params') else 'N/A'
    }

# Função principal para processar os modelos
def test_models(original_data_path, synthetic_data_path, repetitions=30, n_splits=5):
    original_datasets = load_datasets(original_data_path)
    synthetic_datasets = load_datasets(synthetic_data_path)

    if not original_datasets:
        print("Nenhum dataset original encontrado.")
        return
    if not synthetic_datasets:
        print("Nenhum dataset sintético encontrado.")
        return

    models = {
        'LinearRegression': LinearRegression(),
        'Ridge': Ridge(),
        'RandomForest': RandomForestRegressor(),
    }

    results = {}

    for file, test_data in tqdm(original_datasets.items(), desc="Processando datasets originais"):
        train_data = synthetic_datasets.get(file)  # Busca pelo mesmo nome do arquivo

        # Verificar se o dataset sintético correspondente foi encontrado
        if train_data is None:
            print(f"Dataset sintético correspondente para {file} não encontrado. Pulando...")
            continue

        # Mostrar colunas para debug em caso de inconsistência
        if train_data.shape[1] != test_data.shape[1]:
            print(f"Dimensão inconsistente entre {file} nos dados sintéticos e originais.")
            print(f"Colunas nos dados originais: {list(test_data.columns)}")
            print(f"Colunas nos dados sintéticos: {list(train_data.columns)}")
            
            # Tentar ajustar colunas automaticamente
            common_columns = test_data.columns.intersection(train_data.columns)
            if len(common_columns) > 0:
                print(f"Ajustando para usar apenas colunas em comum: {list(common_columns)}")
                train_data = train_data[common_columns]
                test_data = test_data[common_columns]
            else:
                print(f"Não foi possível ajustar {file}. Pulando...")
                continue

        # Log de tamanhos dos datasets
        print(f"Processando {file} - Treino: {train_data.shape[0]} amostras, Validação: {test_data.shape[0]} amostras")

        if train_data.shape[0] < 5 or test_data.shape[0] < 2:
            print(f"Dataset {file} insuficiente para processamento.")
            continue

        # Processar modelos em paralelo
        model_results = Parallel(n_jobs=-1)(
            delayed(process_model_kfold)(train_data, test_data, model_name, model, repetitions, n_splits)
            for model_name, model in models.items()
        )

        results[file] = {model_name: result for model_name, result in zip(models.keys(), model_results)}

    # Salvar resultados em um Excel
    for file, result in results.items():
        rows = []
        for model_name, metrics in result.items():
            row = metrics.copy()
            row['model'] = model_name
            rows.append(row)

        result_df = pd.DataFrame(rows)
        output_file = os.path.splitext(file)[0] + '_resultados_kfold.xlsx'
        output_path = os.path.join(synthetic_data_path, output_file)
        try:
            result_df.to_excel(output_path, index=False)
            print(f"Resultados para {file} salvos em {output_path}")
        except Exception as e:
            print(f"Erro ao salvar resultados para {file}: {e}")

# Caminhos para os datasets
original_data_path = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\correlacao\1-tratamento'
synthetic_data_path = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\correlacao\3-simulacao\1-tratamento'

# Executar o processo
test_models(original_data_path, synthetic_data_path, repetitions=30, n_splits=5)


Processando datasets originais:   0%|          | 0/4 [00:00<?, ?it/s]

Dimensão inconsistente entre tratamento_china.txt nos dados sintéticos e originais.
Colunas nos dados originais: ['ID', 'AFP', 'Output', 'Enquiry', 'Interface', 'Changed', 'Deleted', 'PDR_AFP', 'Resource', 'Dev.Type', 'Duration', 'N_effort', 'Effort']
Colunas nos dados sintéticos: ['ID', 'AFP', 'Input', 'Output', 'Enquiry', 'File', 'Interface', 'Added', 'Changed', 'Deleted', 'PDR_AFP', 'Resource', 'Dev.Type', 'Duration', 'N_effort', 'Effort']
Ajustando para usar apenas colunas em comum: ['ID', 'AFP', 'Output', 'Enquiry', 'Interface', 'Changed', 'Deleted', 'PDR_AFP', 'Resource', 'Dev.Type', 'Duration', 'N_effort', 'Effort']
Processando tratamento_china.txt - Treino: 1250 amostras, Validação: 499 amostras


Processando datasets originais:  25%|██▌       | 1/4 [02:25<07:15, 145.19s/it]

Dimensão inconsistente entre tratamento_cocomo81.txt nos dados sintéticos e originais.
Colunas nos dados originais: ['rely', 'data', 'cplx', 'time', 'stor', 'virt', 'turn', 'acap', 'aexp', 'pcap', 'vexp', 'lexp', 'modp', 'tool', 'sced', 'loc', 'actual']
Colunas nos dados sintéticos: ['rely', 'data', 'virt', 'acap', 'pcap', 'modp', 'actual']
Ajustando para usar apenas colunas em comum: ['rely', 'data', 'virt', 'acap', 'pcap', 'modp', 'actual']
Processando tratamento_cocomo81.txt - Treino: 1250 amostras, Validação: 63 amostras


Processando datasets originais:  25%|██▌       | 1/4 [02:42<08:06, 162.03s/it]


KeyboardInterrupt: 

# Dados sintéticos | usa dados sintéticos com dados originais para treinamento e para teste usa apenas os dados originais.

usa 5 folds, com dados originais e sintéticos. usa 1 fold para teste com dados originais sempre. Os outros 4 folds usam uma mescla de dados sintéticos e originais juntos. Mesma quantidade de variáveis.

In [6]:
import os
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np
from tqdm import tqdm

# Função para carregar datasets
def load_datasets(path):
    datasets = {}
    for root, _, files in os.walk(path):
        for file in files:
            if file.endswith('.txt'):
                file_path = os.path.join(root, file)
                try:
                    datasets[file] = pd.read_csv(file_path, delimiter=None, engine='python')
                except Exception as e:
                    print(f"Erro ao carregar {file}: {e}")
    return datasets

# Função para processar o modelo com KFold adaptado
def process_kfold_with_synthetic(original_data, synthetic_data, models, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    metrics = {model_name: {'mse': [], 'rmse': [], 'r2': [], 'mae': []} for model_name in models.keys()}
    best_params = {}

    # Divisão dos dados originais em folds
    X_original = original_data.iloc[:, :-1].values
    y_original = original_data.iloc[:, -1].values
    X_synthetic = synthetic_data.iloc[:, :-1].values
    y_synthetic = synthetic_data.iloc[:, -1].values

    for fold_idx, (train_index, test_index) in enumerate(kf.split(X_original), 1):
        # Dados de teste (fold atual)
        X_fold_test, y_fold_test = X_original[test_index], y_original[test_index]

        # Dados de treino (sintéticos + outros folds de dados originais)
        X_fold_train, y_fold_train = X_original[train_index], y_original[train_index]
        X_train_combined = np.vstack([X_synthetic, X_fold_train])
        y_train_combined = np.concatenate([y_synthetic, y_fold_train])

        # Avaliar cada modelo
        for model_name, model in models.items():
            try:
                model.fit(X_train_combined, y_train_combined)
                y_pred = model.predict(X_fold_test)

                # Métricas de avaliação
                mse = mean_squared_error(y_fold_test, y_pred)
                rmse = np.sqrt(mse)
                r2 = r2_score(y_fold_test, y_pred)
                mae = mean_absolute_error(y_fold_test, y_pred)

                # Armazenar métricas
                metrics[model_name]['mse'].append(mse)
                metrics[model_name]['rmse'].append(rmse)
                metrics[model_name]['r2'].append(r2)
                metrics[model_name]['mae'].append(mae)

                # Salvar hiperparâmetros
                best_params[model_name] = model.get_params()
            except Exception as e:
                print(f"Erro no modelo {model_name} no Fold {fold_idx}: {e}")

    # Calcular métricas médias para cada modelo
    final_metrics = {
        model_name: {
            'mse': np.mean(metrics[model_name]['mse']),
            'rmse': np.mean(metrics[model_name]['rmse']),
            'r2': np.mean(metrics[model_name]['r2']),
            'mae': np.mean(metrics[model_name]['mae'])
        }
        for model_name in models.keys()
    }

    return final_metrics, best_params

# Função principal
def test_models_with_synthetic_and_kfold(original_data_path, synthetic_data_path, n_splits=5, test_size=0.2, repetitions=30):
    original_datasets = load_datasets(original_data_path)
    synthetic_datasets = load_datasets(synthetic_data_path)

    if not original_datasets:
        print("Nenhum dataset original encontrado.")
        return
    if not synthetic_datasets:
        print("Nenhum dataset sintético encontrado.")
        return

    models = {
        'LinearRegression': LinearRegression(),
        'Ridge': Ridge(),
        'RandomForest': RandomForestRegressor(),
    }

    overall_results = {}
    best_model_params = {}

    for file, original_data in tqdm(original_datasets.items(), desc="Processando datasets originais"):
        synthetic_data = synthetic_datasets.get(file)

        if synthetic_data is None:
            print(f"Dataset sintético correspondente para {file} não encontrado. Pulando...")
            continue

        # Garantir divisão 80/20
        n_test = int(len(original_data) * test_size)
        if n_test < n_splits:
            print(f"Dataset {file} possui amostras insuficientes para dividir em {n_splits} folds com 80/20. Pulando...")
            continue

        # Rodar processamento 30 vezes
        file_results = {model_name: {'mse': [], 'rmse': [], 'r2': [], 'mae': []} for model_name in models.keys()}
        for _ in range(repetitions):
            metrics, best_params = process_kfold_with_synthetic(original_data, synthetic_data, models, n_splits)
            for model_name, metric_values in metrics.items():
                for key in metric_values:
                    file_results[model_name][key].append(metric_values[key])
                best_model_params[model_name] = best_params[model_name]

        # Calcular médias finais das métricas
        overall_results[file] = {
            model_name: {metric: np.mean(values) for metric, values in model_metrics.items()}
            for model_name, model_metrics in file_results.items()
        }

    # Salvar resultados
    for file, metrics in overall_results.items():
        results = []
        for model_name, model_metrics in metrics.items():
            results.append({'Model': model_name, **model_metrics, 'Best Params': best_model_params[model_name]})
        result_df = pd.DataFrame(results)
        output_file = os.path.join(synthetic_data_path, f'{file}_resultados_alisson.xlsx')
        result_df.to_excel(output_file, index=False)
        print(f"Resultados finais salvos em {output_file}")

# Caminhos para os datasets
original_data_path = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\1-tratamento-target-encoding-normalizado'
synthetic_data_path = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao\1-tratamento'

original_data_path2 = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\2-geracao-variaveis\GAN-25'
synthetic_data_path2 = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao\GAN-25'

original_data_path3 = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\2-geracao-variaveis\GAN-50'
synthetic_data_path3 = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao\GAN-50'

original_data_path4 = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\2-geracao-variaveis\GAN-75'
synthetic_data_path4 = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao\GAN-75'

original_data_path5 = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\2-geracao-variaveis\MIMIC-25'
synthetic_data_path5 = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao\MIMIC-25'

original_data_path6 = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\2-geracao-variaveis\MIMIC-50'
synthetic_data_path6 = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao\MIMIC-50'

original_data_path7 = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\2-geracao-variaveis\MIMIC-75'
synthetic_data_path7 = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao\MIMIC-75'

# Executar o processo
test_models_with_synthetic_and_kfold(original_data_path2, synthetic_data_path2, n_splits=5, test_size=0.2, repetitions=30)
test_models_with_synthetic_and_kfold(original_data_path3, synthetic_data_path3, n_splits=5, test_size=0.2, repetitions=30)
test_models_with_synthetic_and_kfold(original_data_path4, synthetic_data_path4, n_splits=5, test_size=0.2, repetitions=30)
test_models_with_synthetic_and_kfold(original_data_path5, synthetic_data_path5, n_splits=5, test_size=0.2, repetitions=30)
test_models_with_synthetic_and_kfold(original_data_path6, synthetic_data_path6, n_splits=5, test_size=0.2, repetitions=30)
test_models_with_synthetic_and_kfold(original_data_path7, synthetic_data_path7, n_splits=5, test_size=0.2, repetitions=30)


Processando datasets originais: 100%|██████████| 4/4 [18:00<00:00, 270.07s/it]


Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao\GAN-25\tratamento_china.txt_resultados_alisson.xlsx
Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao\GAN-25\tratamento_cocomo81.txt_resultados_alisson.xlsx
Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao\GAN-25\tratamento_desharnais.txt_resultados_alisson.xlsx
Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao\GAN-25\tratamento_maxwell.txt_resultados_alisson.xlsx


Processando datasets originais: 100%|██████████| 4/4 [23:05<00:00, 346.40s/it]


Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao\GAN-50\tratamento_china.txt_resultados_alisson.xlsx
Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao\GAN-50\tratamento_cocomo81.txt_resultados_alisson.xlsx
Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao\GAN-50\tratamento_desharnais.txt_resultados_alisson.xlsx
Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao\GAN-50\tratamento_maxwell.txt_resultados_alisson.xlsx


Processando datasets originais: 100%|██████████| 4/4 [24:59<00:00, 374.90s/it]


...

Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao\MIMIC-25\tratamento_china.txt_resultados_alisson.xlsx
Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao\MIMIC-25\tratamento_cocomo81.txt_resultados_alisson.xlsx
Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao\MIMIC-25\tratamento_desharnais.txt_resultados_alisson.xlsx
Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao\MIMIC-25\tratamento_maxwell.txt_resultados_alisson.xlsx


Processando datasets originais: 100%|██████████| 4/4 [23:03<00:00, 345.92s/it]


Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao\MIMIC-50\tratamento_china.txt_resultados_alisson.xlsx
Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao\MIMIC-50\tratamento_cocomo81.txt_resultados_alisson.xlsx
Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao\MIMIC-50\tratamento_desharnais.txt_resultados_alisson.xlsx
Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao\MIMIC-50\tratamento_maxwell.txt_resultados_alisson.xlsx


Processando datasets originais: 100%|██████████| 4/4 [25:08<00:00, 377.11s/it]

Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao\MIMIC-75\tratamento_china.txt_resultados_alisson.xlsx
Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao\MIMIC-75\tratamento_cocomo81.txt_resultados_alisson.xlsx
Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao\MIMIC-75\tratamento_desharnais.txt_resultados_alisson.xlsx
Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao\MIMIC-75\tratamento_maxwell.txt_resultados_alisson.xlsx


# Dados sintéticos | usa dados sintéticos com dados originais para treinamento e para teste usa apenas os dados originais.

usa 5 folds, com dados originais e sintéticos. usa 1 fold para teste com dados originais sempre. Os outros 4 folds usam uma mescla de dados sintéticos e originais juntos. 

# (usa apenas as variáveis presentes em ambos os datasets)

In [4]:
import os
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np
from tqdm import tqdm

# Função para carregar datasets com seleção de variáveis comuns
def load_datasets_with_variable_selection(original_path, synthetic_path):
    original_datasets = {}
    synthetic_datasets = {}

    for root, _, files in os.walk(original_path):
        for file in files:
            if file.endswith('.txt'):
                original_file_path = os.path.join(root, file)
                synthetic_file_path = os.path.join(synthetic_path, file)
                
                if not os.path.exists(synthetic_file_path):
                    print(f"Dataset sintético correspondente para {file} não encontrado. Pulando...")
                    continue
                
                try:
                    # Carregar datasets
                    original_data = pd.read_csv(original_file_path, delimiter=None, engine='python')
                    synthetic_data = pd.read_csv(synthetic_file_path, delimiter=None, engine='python')

                    # Verificar e selecionar variáveis comuns
                    common_columns = original_data.columns.intersection(synthetic_data.columns)
                    if len(common_columns) < 2:  # Pelo menos uma variável + target são esperados
                        print(f"Dataset {file} não possui variáveis comuns suficientes. Pulando...")
                        continue
                    
                    # Manter apenas colunas comuns
                    original_datasets[file] = original_data[common_columns]
                    synthetic_datasets[file] = synthetic_data[common_columns]
                
                except Exception as e:
                    print(f"Erro ao carregar {file}: {e}")
    
    return original_datasets, synthetic_datasets

# Função para processar o modelo com KFold adaptado
def process_kfold_with_synthetic(original_data, synthetic_data, models, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    metrics = {model_name: {'mse': [], 'rmse': [], 'r2': [], 'mae': []} for model_name in models.keys()}
    best_params = {}

    # Divisão dos dados originais em folds
    X_original = original_data.iloc[:, :-1].values
    y_original = original_data.iloc[:, -1].values
    X_synthetic = synthetic_data.iloc[:, :-1].values
    y_synthetic = synthetic_data.iloc[:, -1].values

    for fold_idx, (train_index, test_index) in enumerate(kf.split(X_original), 1):
        # Dados de teste (fold atual)
        X_fold_test, y_fold_test = X_original[test_index], y_original[test_index]

        # Dados de treino (sintéticos + outros folds de dados originais)
        X_fold_train, y_fold_train = X_original[train_index], y_original[train_index]
        X_train_combined = np.vstack([X_synthetic, X_fold_train])
        y_train_combined = np.concatenate([y_synthetic, y_fold_train])

        # Avaliar cada modelo
        for model_name, model in models.items():
            try:
                model.fit(X_train_combined, y_train_combined)
                y_pred = model.predict(X_fold_test)

                # Métricas de avaliação
                mse = mean_squared_error(y_fold_test, y_pred)
                rmse = np.sqrt(mse)
                r2 = r2_score(y_fold_test, y_pred)
                mae = mean_absolute_error(y_fold_test, y_pred)

                # Armazenar métricas
                metrics[model_name]['mse'].append(mse)
                metrics[model_name]['rmse'].append(rmse)
                metrics[model_name]['r2'].append(r2)
                metrics[model_name]['mae'].append(mae)

                # Salvar hiperparâmetros
                best_params[model_name] = model.get_params()
            except Exception as e:
                print(f"Erro no modelo {model_name} no Fold {fold_idx}: {e}")

    # Calcular métricas médias para cada modelo
    final_metrics = {
        model_name: {
            'mse': np.mean(metrics[model_name]['mse']),
            'rmse': np.mean(metrics[model_name]['rmse']),
            'r2': np.mean(metrics[model_name]['r2']),
            'mae': np.mean(metrics[model_name]['mae'])
        }
        for model_name in models.keys()
    }

    return final_metrics, best_params

# Função principal com seleção de variáveis comuns
def test_models_with_selected_variables(original_data_path, synthetic_data_path, n_splits=5, test_size=0.2, repetitions=30):
    original_datasets, synthetic_datasets = load_datasets_with_variable_selection(original_data_path, synthetic_data_path)

    if not original_datasets:
        print("Nenhum dataset original encontrado ou compatível.")
        return
    if not synthetic_datasets:
        print("Nenhum dataset sintético encontrado ou compatível.")
        return

    models = {
        'LinearRegression': LinearRegression(),
        'Ridge': Ridge(),
        'RandomForest': RandomForestRegressor(),
    }

    overall_results = {}
    best_model_params = {}

    for file, original_data in tqdm(original_datasets.items(), desc="Processando datasets originais"):
        synthetic_data = synthetic_datasets.get(file)

        # Garantir divisão 80/20
        n_test = int(len(original_data) * test_size)
        if n_test < n_splits:
            print(f"Dataset {file} possui amostras insuficientes para dividir em {n_splits} folds com 80/20. Pulando...")
            continue

        # Rodar processamento 30 vezes
        file_results = {model_name: {'mse': [], 'rmse': [], 'r2': [], 'mae': []} for model_name in models.keys()}
        for _ in range(repetitions):
            metrics, best_params = process_kfold_with_synthetic(original_data, synthetic_data, models, n_splits)
            for model_name, metric_values in metrics.items():
                for key in metric_values:
                    file_results[model_name][key].append(metric_values[key])
                best_model_params[model_name] = best_params[model_name]

        # Calcular médias finais das métricas
        overall_results[file] = {
            model_name: {metric: np.mean(values) for metric, values in model_metrics.items()}
            for model_name, model_metrics in file_results.items()
        }

    # Salvar resultados
    for file, metrics in overall_results.items():
        results = []
        for model_name, model_metrics in metrics.items():
            results.append({'Model': model_name, **model_metrics, 'Best Params': best_model_params[model_name]})
        result_df = pd.DataFrame(results)
        output_file = os.path.join(synthetic_data_path, f'{file}_resultados_alisson.xlsx')
        result_df.to_excel(output_file, index=False)
        print(f"Resultados finais salvos em {output_file}")

# Caminhos para os datasets
original_data_path1 = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\75\1-tratamento-target-encoding-normalizado'
synthetic_data_path1 = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\75\3-simulacao\1-tratamento'

original_data_path2 = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\75\2-geracao-variaveis\GAN-25'
synthetic_data_path2 = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\75\3-simulacao\GAN-25'

original_data_path3 = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\75\2-geracao-variaveis\GAN-50'
synthetic_data_path3 = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\75\3-simulacao\GAN-50'

original_data_path4 = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\75\2-geracao-variaveis\GAN-75'
synthetic_data_path4 = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\75\3-simulacao\GAN-75'

original_data_path5 = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\75\2-geracao-variaveis\MIMIC-25'
synthetic_data_path5 = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\75\3-simulacao\MIMIC-25'

original_data_path6 = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\75\2-geracao-variaveis\MIMIC-50'
synthetic_data_path6 = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\75\3-simulacao\MIMIC-50'

original_data_path7 = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\75\2-geracao-variaveis\MIMIC-75'
synthetic_data_path7 = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\75\3-simulacao\MIMIC-75'

# Executar o processo com seleção de variáveis comuns
test_models_with_selected_variables(original_data_path1, synthetic_data_path1, n_splits=5, test_size=0.2, repetitions=30)
test_models_with_selected_variables(original_data_path2, synthetic_data_path2, n_splits=5, test_size=0.2, repetitions=30)
test_models_with_selected_variables(original_data_path3, synthetic_data_path3, n_splits=5, test_size=0.2, repetitions=30)
test_models_with_selected_variables(original_data_path4, synthetic_data_path4, n_splits=5, test_size=0.2, repetitions=30)
test_models_with_selected_variables(original_data_path5, synthetic_data_path5, n_splits=5, test_size=0.2, repetitions=30)
test_models_with_selected_variables(original_data_path6, synthetic_data_path6, n_splits=5, test_size=0.2, repetitions=30)
test_models_with_selected_variables(original_data_path7, synthetic_data_path7, n_splits=5, test_size=0.2, repetitions=30)


Processando datasets originais: 100%|██████████| 4/4 [11:38<00:00, 174.51s/it]


Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\75\3-simulacao\1-tratamento\tratamento_china.txt_resultados_alisson.xlsx
Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\75\3-simulacao\1-tratamento\tratamento_cocomo81.txt_resultados_alisson.xlsx
Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\75\3-simulacao\1-tratamento\tratamento_desharnais.txt_resultados_alisson.xlsx
Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\75\3-simulacao\1-tratamento\tratamento_maxwell.txt_resultados_alisson.xlsx


Processando datasets originais: 100%|██████████| 4/4 [13:15<00:00, 198.96s/it]


Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\75\3-simulacao\GAN-25\tratamento_china.txt_resultados_alisson.xlsx
Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\75\3-simulacao\GAN-25\tratamento_cocomo81.txt_resultados_alisson.xlsx
Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\75\3-simulacao\GAN-25\tratamento_desharnais.txt_resultados_alisson.xlsx
Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\75\3-simulacao\GAN-25\tratamento_maxwell.txt_resultados_alisson.xlsx


Processando datasets originais: 100%|██████████| 4/4 [16:56<00:00, 254.20s/it]


...

Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\75\3-simulacao\MIMIC-25\tratamento_china.txt_resultados_alisson.xlsx
Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\75\3-simulacao\MIMIC-25\tratamento_cocomo81.txt_resultados_alisson.xlsx
Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\75\3-simulacao\MIMIC-25\tratamento_desharnais.txt_resultados_alisson.xlsx
Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\75\3-simulacao\MIMIC-25\tratamento_maxwell.txt_resultados_alisson.xlsx


Processando datasets originais: 100%|██████████| 4/4 [15:37<00:00, 234.31s/it]


Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\75\3-simulacao\MIMIC-50\tratamento_china.txt_resultados_alisson.xlsx
Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\75\3-simulacao\MIMIC-50\tratamento_cocomo81.txt_resultados_alisson.xlsx
Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\75\3-simulacao\MIMIC-50\tratamento_desharnais.txt_resultados_alisson.xlsx
Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\75\3-simulacao\MIMIC-50\tratamento_maxwell.txt_resultados_alisson.xlsx


Processando datasets originais: 100%|██████████| 4/4 [16:14<00:00, 243.51s/it]

Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\75\3-simulacao\MIMIC-75\tratamento_china.txt_resultados_alisson.xlsx
Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\75\3-simulacao\MIMIC-75\tratamento_cocomo81.txt_resultados_alisson.xlsx
Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\75\3-simulacao\MIMIC-75\tratamento_desharnais.txt_resultados_alisson.xlsx
Resultados finais salvos em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\75\3-simulacao\MIMIC-75\tratamento_maxwell.txt_resultados_alisson.xlsx
